### Analysis for Wristband Project

Aug/14/2023


## Import Dependency

In [1]:
import numpy as np
import pandas as pd

## Import Data

Load Data into an Array List

In [27]:
# subject name to get data path

subject_name = "Hokin"
data_path = "data/" + subject_name
section_numbers = 4

open_data = []
close_data = []
rest_data = []

for i in range (section_numbers):
    section_path = data_path + "/Section_Number_" + str(i)
    open_df = pd.read_csv(section_path + "/open.csv")
    open_data.append(open_df.values)
    close_df = pd.read_csv(section_path + "/close.csv")
    close_data.append(close_df.values)
    rest_df = pd.read_csv(section_path + "/rest.csv")
    rest_data.append(rest_df.values)

Checking if everything makes sense at this point

In [28]:
print(len(rest_data))

4


In [30]:
print(type(open_data))

<class 'list'>


In [31]:
print(np.shape(open_data[2]))

(2000, 21)


In [32]:
print(np.shape(rest_data[1]))

(18024, 21)
